In [2]:
import h2o
import pandas as pd
import zipfile
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
def cargar_dataset(ruta_zip, nombre_tsv):
    with zipfile.ZipFile(ruta_zip, 'r') as archivo_zip:
        with archivo_zip.open(nombre_tsv) as archivo_tsv:
            dataset = pd.read_csv(archivo_tsv, sep='\t')
    return dataset

def preprocesar_dataset(dataset):
    dataset['name'] = dataset['name'].astype('str')
    dataset['comment'] = dataset['comment'].astype('str')
    return dataset

In [4]:
df = cargar_dataset('../../data/RepoRT_classified_CCinformation.zip', 'RepoRT_classified_CCinformation.tsv')
df = preprocesar_dataset(df)

C:\Users\cmora\AppData\Local\Temp\ipykernel_36156\590027052.py:4: DtypeWarning: Columns (1,13) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(archivo_tsv, sep='\t')


In [5]:
# Creamos la columna n_experimento: representa en el experimento donde se realizó la toma de datos.
df['n_experimento'] = df['id'].str.split('_').str[0]

df['n_experimento'] = df['n_experimento'].astype(int)

Voy a dejar las columnas usp_columns en one-hot

In [6]:
usp_columns = [
    'column.usp.code_0', 'column.usp.code_L1', 'column.usp.code_L10',
    'column.usp.code_L109', 'column.usp.code_L11', 'column.usp.code_L114',
    'column.usp.code_L122', 'column.usp.code_L3', 'column.usp.code_L43',
    'column.usp.code_L68', 'column.usp.code_L7'
]

df[usp_columns]

,column.usp.code_0,column.usp.code_L1,column.usp.code_L10,column.usp.code_L109,column.usp.code_L11,column.usp.code_L114,column.usp.code_L122,column.usp.code_L3,column.usp.code_L43,column.usp.code_L68,column.usp.code_L7
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
164341,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
164342,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
164343,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
164344,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Eliminamos las columnas que tienen que ver con el gradiente

In [7]:
# Paso 1: Seleccionar las columnas que coincidan con el patrón 'eluent.1.*\s[1-9]\d*'
columnas_a_eliminar = df.filter(regex=r'eluent\.1.*\s[1-9]\d*').columns
print("Columnas 'eluent.1' a eliminar:", columnas_a_eliminar)

# Paso 2: Seleccionar las columnas que coincidan con el patrón 'eluent.2.*\s[1-9]\d*'
columnas_a_eliminar = columnas_a_eliminar.append(df.filter(regex=r'eluent\.2.*\s[1-9]\d*').columns)
print("Columnas 'eluent.2' a eliminar:", columnas_a_eliminar)

# Paso 3: Seleccionar las columnas que coincidan con los patrones 'flowrate' o 't' seguidas de un número distinto de 0
columnas_a_eliminar = columnas_a_eliminar.append(df.filter(regex=r'^(flow_rate|t)\s[1-9]\d*$').columns)
print("Columnas 'flowrate' o 't' a eliminar:", columnas_a_eliminar)

# Paso 4: Seleccionar las columnas 'eluent.1.pH 0' y 'eluent.2.pH 0'
columnas_pH = df.filter(regex=r'eluent\.[12]\.pH\s0').columns
columnas_a_eliminar = columnas_a_eliminar.append(columnas_pH)
print("Columnas 'eluent.1.pH 0' y 'eluent.2.pH 0' a eliminar:", columnas_pH)

# Paso 5: Verificar que las columnas a eliminar no sean vacías
if not columnas_a_eliminar.empty:
    # Crear un nuevo DataFrame sin esas columnas
    df = df.drop(columns=columnas_a_eliminar)
    print("Nuevo DataFrame sin las columnas a eliminar.")
else:
    print("No hay columnas que eliminar según los patrones especificados.")

Columnas 'eluent.1' a eliminar: Index(['eluent.1.h2o 1', 'eluent.1.meoh 1', 'eluent.1.acn 1',
       'eluent.1.iproh 1', 'eluent.1.acetone 1', 'eluent.1.hex 1',
       'eluent.1.chcl3 1', 'eluent.1.ch2cl2 1', 'eluent.1.hept 1',
       'eluent.1.formic 1',
       ...
       'eluent.1.nh4bicarb 17', 'eluent.1.nh4f 17', 'eluent.1.nh4oh 17',
       'eluent.1.trieth 17', 'eluent.1.triprop 17', 'eluent.1.tribut 17',
       'eluent.1.nndimethylhex 17', 'eluent.1.medronic 17', 'eluent.1.pH 17',
       'eluent.1 17'],
      dtype='object', length=442)
Columnas 'eluent.2' a eliminar: Index(['eluent.1.h2o 1', 'eluent.1.meoh 1', 'eluent.1.acn 1',
       'eluent.1.iproh 1', 'eluent.1.acetone 1', 'eluent.1.hex 1',
       'eluent.1.chcl3 1', 'eluent.1.ch2cl2 1', 'eluent.1.hept 1',
       'eluent.1.formic 1',
       ...
       'eluent.2.nh4bicarb 17', 'eluent.2.nh4f 17', 'eluent.2.nh4oh 17',
       'eluent.2.trieth 17', 'eluent.2.triprop 17', 'eluent.2.tribut 17',
       'eluent.2.nndimethylhex 17', '

Creamos 2 columnas: eluyente1 y eluyente2

In [8]:
# Función para crear las columnas 'eluyente1' y 'eluyente2'
def agregar_eluyentes(df):
    def obtener_eluyente(row, eluyente_col_prefix):
        # Extraer los nombres de los compuestos que tienen valor mayor que 0
        compuestos = []
        for col in df.columns:
            # Verificar que la columna empieza con eluent.1 o eluent.2 y que el valor es mayor que 0
            if col.startswith(eluyente_col_prefix) and row[col] > 0:
                # Intentar extraer el nombre del compuesto desde la columna
                try:
                    compuesto = col.split('.')[2]  # Tercera parte es el nombre del compuesto
                    compuesto = compuesto.split(" ")[0]  # Quedarse solo con la primera parte antes del " 0"
                    compuestos.append(compuesto)
                except IndexError:
                    # Si ocurre un error (por ejemplo, si el formato no es el esperado), saltar esta columna
                    pass
        return ', '.join(compuestos)
    
    # Crear las nuevas columnas 'eluyente1' y 'eluyente2'
    df['eluyente1'] = df.apply(lambda row: obtener_eluyente(row, 'eluent.1'), axis=1)
    df['eluyente2'] = df.apply(lambda row: obtener_eluyente(row, 'eluent.2'), axis=1)
    
    return df

In [9]:
df = agregar_eluyentes(df)

In [10]:
with open("../../tmp/nombres_columnas.txt", "w") as f:
    f.write(str(df.columns.to_list()))


Más columnas a eliminar

In [11]:
columnas_a_eliminar = [
    "comment", "classyfire.level6", "classyfire.level5", "classyfire.subclass", "classyfire.superclass", "classyfire.kingdom",
    "column.usp.code_0","column.usp.code_L1", "column.usp.code_L10", "column.usp.code_L109", "column.usp.code_L11", "column.usp.code_L114", "column.usp.code_L122", "column.usp.code_L3", "column.usp.code_L43", "column.usp.code_L68", "column.usp.code_L7",
    "missing_values",
    "t 0", "flow_rate 0",
    "eluent.1.h2o 0", "eluent.1.meoh 0", "eluent.1.acn 0", "eluent.1.iproh 0", "eluent.1.acetone 0", "eluent.1.hex 0", "eluent.1.chcl3 0", "eluent.1.ch2cl2 0", "eluent.1.hept 0", "eluent.1.formic 0", "eluent.1.acetic 0", "eluent.1.trifluoroacetic 0", "eluent.1.phosphor 0", "eluent.1.nh4ac 0", "eluent.1.nh4form 0", "eluent.1.nh4carb 0", "eluent.1.nh4bicarb 0", "eluent.1.nh4f 0", "eluent.1.nh4oh 0", "eluent.1.trieth 0", "eluent.1.triprop 0", "eluent.1.tribut 0", "eluent.1.nndimethylhex 0", "eluent.1.medronic 0", "eluent.2.h2o 0", "eluent.2.meoh 0", "eluent.2.acn 0", "eluent.2.iproh 0", "eluent.2.acetone 0", "eluent.2.hex 0", "eluent.2.chcl3 0", "eluent.2.ch2cl2 0", "eluent.2.hept 0", "eluent.2.formic 0", "eluent.2.acetic 0", "eluent.2.trifluoroacetic 0", "eluent.2.phosphor 0", "eluent.2.nh4ac 0", "eluent.2.nh4form 0", "eluent.2.nh4carb 0", "eluent.2.nh4bicarb 0", "eluent.2.nh4f 0", "eluent.2.nh4oh 0", "eluent.2.trieth 0", "eluent.2.triprop 0", "eluent.2.tribut 0", "eluent.2.nndimethylhex 0", "eluent.2.medronic 0", "eluent.1 0", "eluent.2 0",
    "name", "id", "inchikey.std", "inchi.std", "formula"
]

df = df.drop(columns=columnas_a_eliminar)

A continuación se va a eliminar smiles.std, pero si se quiere contar con la estructura de los metabolitos en un futuro podemos utilizar la siguiente celda

In [ ]:
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors
# Convertir la columna SMILES en moléculas RDKit
df = df[df["smiles.std"].notnull()]
df["Mol"] = df["smiles.std"].apply(lambda x: Chem.MolFromSmiles(x) if pd.notnull(x) else None)

#df["Mol"] = df["smiles.std"].apply(lambda x: Chem.MolFromSmarts(x) if x else None) #<- OTRA FORMA DE HACERLO

In [12]:
df = df.drop(columns=['smiles.std'])

Fusionamos las columnas classyfire.class y alternative_parents

In [13]:
# Asegurar que las columnas son strings y no NaN
df["classyfire.class"] = df["classyfire.class"].fillna("").astype(str)
df["alternative_parents"] = df["alternative_parents"].fillna("").astype(str)

# Dividir las categorías en listas (asumiendo que están separadas por comas)
df["familias_metabolitos"] = df["classyfire.class"].str.split(", ") + df["alternative_parents"].str.split(", ")

# Eliminar duplicados dentro de cada lista
df["familias_metabolitos"] = df["familias_metabolitos"].apply(lambda x: list(set(x)))

# Ver resultados
print(df[["classyfire.class", "alternative_parents", "familias_metabolitos"]].head())

                                    classyfire.class  \
0                     Flavonoids (CHEMONTID:0000334)   
1                     Flavonoids (CHEMONTID:0000334)   
2                     Flavonoids (CHEMONTID:0000334)   
3  Benzene and substituted derivatives (CHEMONTID...   
4  Benzene and substituted derivatives (CHEMONTID...   

                                 alternative_parents  \
0  Organic compounds, Organoheterocyclic compound...   
1  Organic compounds, Organoheterocyclic compound...   
2  Organic compounds, Organoheterocyclic compound...   
3  Organic compounds, Phenols, Resorcinols, Benzo...   
4  Organic compounds, Phenols, Benzoic acids and ...   

                                familias_metabolitos  
0  [Phenols, Chemical entities, Organoheterocycli...  
1  [Phenols, Chemical entities, Organoheterocycli...  
2  [Phenols, Chemical entities, Organoheterocycli...  
3  [Phenols, Chemical entities, Benzene and subst...  
4  [Phenols, Benzene and substituted derivatives ..

Borramos las columnas classyfire.class y alternative_parents

In [14]:
df = df.drop(columns=["classyfire.class", "alternative_parents"])

Codificamos la columna de familias

In [15]:

import numpy as np

In [16]:

from gensim.models import Word2Vec


# Entrenar un modelo Word2Vec con las familias de metabolitos
model = Word2Vec(sentences=df["familias_metabolitos"], vector_size=10, window=5, min_count=1, workers=4)

# Función para convertir una lista de familias en un vector promedio
def vectorizar_familias(familias):
    vectors = [model.wv[f] for f in familias if f in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(10)  # 10 es el tamaño del embedding

# Aplicar la vectorización a la columna
df["familias_vector"] = df["familias_metabolitos"].apply(vectorizar_familias)

# Convertir la lista de vectores en columnas separadas
familia_embeddings = pd.DataFrame(df["familias_vector"].to_list(), columns=[f"familia_emb_{i}" for i in range(10)])

# Concatenar con el DataFrame original
df = pd.concat([df, familia_embeddings], axis=1).drop(columns=["familias_vector"])

print(df.head())


       rt                 column.name  column.length  column.id  \
0  12.070  Waters ACQUITY UPLC HSS T3          150.0        2.1   
1  15.990  Waters ACQUITY UPLC HSS T3          150.0        2.1   
2   8.675  Waters ACQUITY UPLC HSS T3          150.0        2.1   
3   7.750  Waters ACQUITY UPLC HSS T3          150.0        2.1   
4  10.600  Waters ACQUITY UPLC HSS T3          150.0        2.1   

   column.particle.size  column.temperature  column.flowrate  column.t0  \
0                   1.8                40.0             0.28    1.18125   
1                   1.8                40.0             0.28    1.18125   
2                   1.8                40.0             0.28    1.18125   
3                   1.8                40.0             0.28    1.18125   
4                   1.8                40.0             0.28    1.18125   

   n_experimento    eluyente1  ... familia_emb_0 familia_emb_1  familia_emb_2  \
0              1  h2o, formic  ...     -0.647996     -2.695779   

In [22]:
df.columns

Index(['rt', 'column.name', 'column.length', 'column.id',
       'column.particle.size', 'column.temperature', 'column.flowrate',
       'column.t0', 'n_experimento', 'eluyente1', 'eluyente2',
       'familias_metabolitos', 'familia_emb_0', 'familia_emb_1',
       'familia_emb_2', 'familia_emb_3', 'familia_emb_4', 'familia_emb_5',
       'familia_emb_6', 'familia_emb_7', 'familia_emb_8', 'familia_emb_9'],
      dtype='object')

In [23]:
df = df.drop(columns=["familias_metabolitos"])


In [18]:
def guardar_como_excel(df, nombre_archivo="../../tmp/excel_1.xlsx", hoja='Sheet1'):
    try:
        df.to_excel(nombre_archivo, sheet_name=hoja, index=False)
        print(f"Archivo guardado exitosamente como '{nombre_archivo}'")
    except Exception as e:
        print(f"Error al guardar el archivo: {e}")

In [25]:
guardar_como_excel(df,"../../tmp/excel_4.xlsx")

Archivo guardado exitosamente como '../../tmp/excel_4.xlsx'
